In [1]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow
from ultralytics import RTDETR

# Check available GPUs
num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  # Example: [0,1,2,3] if 4 GPUs are available

# Define dataset path (update if needed)
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos 6"
data_yaml = os.path.join(dataset_path, "data.yaml")

# Check if dataset exists
if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="4cCp50rIjAm1ej80fF0P")
    project = rf.workspace("eyes-final-final").project("aptos-final")
    version = project.version(2)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

# Load YOLO model
model = RTDETR("rtdetr-l")  # This ensures it loads RT-DETR correctly




Dataset already exists. Using the existing dataset.


In [2]:
import os
from collections import defaultdict

# Base dataset path
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos 6"

# Corrected label paths
labels_dirs = [
    os.path.join(dataset_path, "valid", "labels"),
    os.path.join(dataset_path, "test", "labels"),
]

# Dictionary to track images per class
class_to_images = defaultdict(set)

# Iterate and count
for labels_dir in labels_dirs:
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                class_to_images[class_id].add(label_file)

# Output counts
print("\nNumber of images per class:")
for class_id in range(5):  # Classes 0 to 4
    count = len(class_to_images[class_id])
    print(f"Class {class_id}: {count} images")


Scanning: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos 6/valid/labels
Scanning: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos 6/test/labels

Number of images per class:
Class 0: 444 images
Class 1: 348 images
Class 2: 371 images
Class 3: 264 images
Class 4: 226 images


In [4]:
# Train with optimized settings
model.train(
    data=data_yaml,
    name="aptos aug 6",
    device=device_ids,  # Use all available GPUs
    epochs=100, 
    batch=32,  # Adjust batch size for better GPU utilization
#     workers=16,  # Use multiple data loading workers
#     optimizer="SGD"  # Use SGD optimizer
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos 6/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=8, project=None, name=aptos aug 66, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, 

train: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos 6/train/labels.cache... 13208 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13208/13208 [00:00<?, ?it/s]
val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos 6/valid/labels.cache... 1238 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1238/1238 [00:00<?, ?it/s]


Plotting labels to runs/detect/aptos aug 66/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 143 weight(decay=0.0), 206 weight(decay=0.0005), 226 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 32 dataloader workers
Logging results to runs/detect/aptos aug 66
Starting training for 100 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/413 [00:00<?, ?it/s]/home/mansi/tfsumit/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:83.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/mansi/tfsumit/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Cont

                   all       1238       1238      0.523      0.435      0.353      0.346

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

                   all       1238       1238      0.976      0.977      0.972      0.972

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/413 [00:00<?, ?it/s]/home/mansi/tfsumit/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:83.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/mansi/tfsumit/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Cont

                   all       1238       1238      0.976      0.977      0.972      0.972

100 epochs completed in 6.054 hours.
Optimizer stripped from runs/detect/aptos aug 66/weights/last.pt, 66.2MB
Optimizer stripped from runs/detect/aptos aug 66/weights/best.pt, 66.2MB

Validating runs/detect/aptos aug 66/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
rt-detr-l summary: 302 layers, 31,994,015 parameters, 0 gradients, 103.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 78/78 [00:20<00:00,  3.84it/s]


                   all       1238       1238      0.978      0.979      0.978      0.978
                     0        331        331      0.997      0.991      0.995      0.995
                     1        261        261      0.974      0.996      0.994      0.994
                     2        279        279      0.997      0.961      0.965      0.965
                     3        198        198       0.95       0.99      0.986      0.986
                     4        169        169      0.972      0.959      0.947      0.947
Speed: 0.2ms preprocess, 8.4ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/aptos aug 66


In [5]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
rt-detr-l summary: 302 layers, 31,994,015 parameters, 0 gradients, 103.5 GFLOPs


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos 6/valid/labels.cache... 1238 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1238/1
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:25<00:00,  1.50it/s]


                   all       1238       1238      0.979      0.979      0.978      0.978
                     0        331        331      0.997      0.991      0.995      0.995
                     1        261        261      0.974      0.996      0.994      0.994
                     2        279        279      0.997      0.961      0.965      0.965
                     3        198        198      0.954       0.99      0.986      0.986
                     4        169        169      0.972      0.959      0.948      0.947
Speed: 0.2ms preprocess, 15.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/aptos aug 662
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3a10734e20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-

In [ ]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)